In [1]:
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import math

In [2]:
json_request = {
  "allocation_weights": [0.3,0.4,0.3],
  "codelist": ["ABAQ","ABQI","ABQX"],
  "benchmark": "ABAQ",
  "initial_amount": 10000,
  "start_date": "2019-03-13",
  "end_date": "2020-03-13",
  "rebalance": True,
  "rebalance_frequency": "Monthly"
}

In [3]:
# Backtesting Portfolio

# Define portfolio, start_date, end_date

allocation_weights = json_request['allocation_weights']
initial_amount = json_request['initial_amount']
start_date = json_request['start_date']
end_date = json_request['end_date']
codelist = json_request['codelist']
benchmark = json_request['benchmark']

rebalance = json_request['rebalance']
rebalance_frequency = json_request['rebalance_frequency']

with open('data/key.txt', 'r') as file:
    api_key = file.read()

In [4]:
indexdata = pd.DataFrame()

quandl_code = codelist
if benchmark != 'None':
    quandl_code = codelist + [benchmark]

for x in range(0,len(quandl_code)):
    quandl_request = (
    'https://www.quandl.com/api/v3/datasets/NASDAQOMX/'
    f'{quandl_code[x]}?start_date={start_date}&end_date={end_date}&api_key={api_key}'
    )
    
    response = requests.get(quandl_request).json()
    
    response_df = pd.DataFrame(response['dataset']['data'])
    response_df = response_df[[0,1]]
    if x == len(quandl_code)-1:
        response_df.columns = ['date','benchmark']
    else:
        response_df.columns = ['date',quandl_code[x]]
    
    if x == 0:
        indexdata = indexdata.append(response_df)
    else:
        indexdata = pd.merge(indexdata,response_df)

In [5]:
month_frequency = {
    'Yearly': 12,
    'Quarterly': 3,
    'Monthly': 1
}

In [6]:
end_date = datetime.strptime(end_date,'%Y-%m-%d')
start_date = datetime.strptime(start_date,'%Y-%m-%d')
num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
if (end_date.day < start_date.day):
    num_months = num_months - 1

In [7]:
event_count = int(math.floor(num_months / month_frequency[rebalance_frequency]))

rebalance_list =[start_date.strftime("%Y-%m-%d")]
rebalance_date = start_date
for x in range(0,event_count):
    rebalance_date = rebalance_date + relativedelta(months=+month_frequency[rebalance_frequency])
    rebalance_list.append(rebalance_date.strftime("%Y-%m-%d"))

In [8]:
asset_projection = []

asset_value = [x * initial_amount for x in allocation_weights]

In [9]:
indexdata

,date,ABAQ,ABQI,ABQX,benchmark
0,2020-03-13,245.78,1704.43,2137.43,245.78
1,2020-03-12,224.39,1554.21,1948.21,224.39
2,2020-03-11,245.22,1694.39,2122.16,245.22
3,2020-03-10,261.19,1807.17,2263.39,261.19
4,2020-03-09,250.82,1731.57,2168.70,250.82
...,...,...,...,...,...
249,2019-03-19,332.06,2315.03,2828.16,332.06
250,2019-03-18,340.11,2374.76,2901.14,340.11
251,2019-03-15,336.45,2346.99,2867.17,336.45
252,2019-03-14,336.27,2346.24,2866.13,336.27


In [10]:
for x in range(0,len(rebalance_list)-1):
    asset_value = [x * initial_amount for x in allocation_weights]
    
    index_subset = indexdata[indexdata['date'] >= rebalance_list[0+x]][indexdata['date'] < rebalance_list[1+x]]
    index_subset = index_subset.sort_values(by='date').reset_index(drop=True)

    allocation = pd.DataFrame()

    for i in range(0,len(allocation_weights)):
        indexcode = index_subset.columns[i+1]
        scaling_factor = asset_value[i] / index_subset[indexcode][0]
        allocation[f'allocation {i}'] = index_subset[indexcode] * scaling_factor
    
    asset_projection.append(allocation)
    
    asset_value = allocation.iloc[len(allocation)-1].sum()

In [11]:
asset_projection = pd.concat(asset_projection).reset_index(drop=True)

In [12]:
asset_projection['output'] = asset_projection.sum(axis = 1)
asset_projection['date'] = indexdata['date']

In [ ]:
if benchmark != 'None':
    scaling_factor = initial_amount / indexdata.sort_values(by='date').reset_index(drop=True)['benchmark'][0]
    asset_projection['benchmark'] = indexdata.sort_values(by='date').reset_index(drop=True)['benchmark'] * scaling_factor

In [21]:
output_field = ['date','output']

if benchmark != 'None':
    output_field = output_field +['benchmark']

output_json = asset_projection[output_field].to_json(orient='columns')

In [23]:
input_json = asset_projection[['date','output']].to_json(orient='columns')

In [49]:
#def calculate metrics(input_json):
input_index = pd.read_json(input_json)
start_end_ratio =  input_index['output'][len(input_index)-1] / input_index['output'][0]
no_years =  input_index['date'][0] - input_index['date'][len(input_index)-1]
no_years = (no_years.days / 365)
CAGR = (start_end_ratio) ** (1/no_years) - 1

In [66]:
meansubtracted_values = input_index['output'] - input_index['output'].mean()
meansubtracted_squared_sum = (meansubtracted_values**2).sum()
variance = meansubtracted_squared_sum / (len(input_index['output'])-1)
stddev = variance ** (1/2)